In [2]:
!pip install paddlex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 952kB 11.3MB/s eta 0:00:01
     |████████████████████████████████| 153kB 67.8MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 31.9MB/s eta 0:00:01
     |████████████████████████████████| 296kB 60.9MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 54.3MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 40.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 31.2MB/s eta 0:00:01
     |████████████████████████████████| 286kB 60.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 50.4MB/s ta 0:00:01
     |████████████████████████████████| 102kB 44.8MB/s ta 0:00:01
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593872 sha256=cf64f757cb2c6c35baca8929a5821fd6d4fea0dab439899caac29d978a09aed5
  Stored in directory: /home/aistudio/.cache/pip/wheels/66/b9/1a/5c513d0b33edd38e4b95052909201336e6526c65226044faff

In [3]:
!unzip data/data114289//train_clean.zip -d work

Archive:  data/data114289//train_clean.zip
   creating: work/angry/
  inflating: work/angry/im0.png      
  inflating: work/angry/im1.png      
  inflating: work/angry/im10.png     
  inflating: work/angry/im100.png    
  inflating: work/angry/im1000.png   
  inflating: work/angry/im1001.png   
  inflating: work/angry/im1002.png   
  inflating: work/angry/im1003.png   
  inflating: work/angry/im1004.png   
  inflating: work/angry/im1005.png   
  inflating: work/angry/im1006.png   
  inflating: work/angry/im1007.png   
  inflating: work/angry/im1008.png   
  inflating: work/angry/im1009.png   
  inflating: work/angry/im101.png    
  inflating: work/angry/im1010.png   
  inflating: work/angry/im1011.png   
  inflating: work/angry/im1012.png   
  inflating: work/angry/im1013.png   
  inflating: work/angry/im1014.png   
  inflating: work/angry/im1015.png   
  inflating: work/angry/im1016.png   
  inflating: work/angry/im1017.png   
  inflating: work/angry/im1018.png   
  inflating: work/an

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
from paddlex import transforms as T
from collections import Counter

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[10-31 00:49:16 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [ ]:
!paddlex --split_dataset --format ImageNet --dataset_dir work --val_value 0.2

# data augment


In [2]:
train_transforms = T.Compose([
                            T.Resize(target_size=(48,48)),
                            T.RandomBlur(), 
                            T.RandomHorizontalFlip(), 
                            T.MixupImage(),
                            T.RandomDistort(),
                            T.Normalize(),
                            ])                        
eval_transforms = T.Compose([T.Resize(target_size=(48,48)),T.Normalize()])


# load dataset

In [4]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='work',
    file_list='work/train_list.txt',
    label_list='work/labels.txt',
    transforms=train_transforms,
    shuffle=True)
print(len(train_dataset)) 
eval_dataset = pdx.datasets.ImageNet(
    data_dir='work/',
    file_list='work/val_list.txt',
    label_list='work/labels.txt',
    transforms=eval_transforms)
print(len(eval_dataset))

2021-10-31 00:49:55 [INFO]	Starting to read file list from dataset...
2021-10-31 00:49:56 [INFO]	22556 samples in file work/train_list.txt
22556
2021-10-31 00:49:56 [INFO]	Starting to read file list from dataset...
2021-10-31 00:49:56 [INFO]	5636 samples in file work/val_list.txt
5636


# train


In [11]:
num_classes = len(train_dataset.labels)
model = pdx.cls.MobileNetV3_large_ssld(num_classes=num_classes)
model.train(num_epochs=20,
            train_dataset=train_dataset,
            train_batch_size=32,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[10, 15, 17],
            save_interval_epochs=1,
            learning_rate=0.0025,
            early_stop=True,
            save_dir='output/MobileNetV3_large_ssld',
            use_vdl=True)

2021-10-30 23:13:20 [INFO]	Downloading MobileNetV3_large_x1_0_ssld_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/MobileNetV3_large_x1_0_ssld_pretrained.pdparams


100%|██████████| 32444/32444 [00:00<00:00, 32915.83KB/s]


2021-10-30 23:13:21 [INFO]	Loading pretrained model from output/MobileNetV3_large_ssld/pretrain/MobileNetV3_large_x1_0_ssld_pretrained.pdparams
2021-10-30 23:13:22 [WARNING]	[SKIP] Shape of pretrained params out.weight doesn't match.(Pretrained: [1280, 1000], Actual: [1280, 6])
2021-10-30 23:13:22 [WARNING]	[SKIP] Shape of pretrained params out.bias doesn't match.(Pretrained: [1000], Actual: [6])
2021-10-30 23:13:23 [INFO]	There are 268/270 variables loaded into MobileNetV3_large_x1_0_ssld.
2021-10-30 23:13:24 [INFO]	[TRAIN] Epoch=1/20, Step=10/704, loss=2.309842, acc1=0.250000, acc5=0.718750, lr=0.002500, time_each_step=0.1s, eta=0:22:34
2021-10-30 23:13:25 [INFO]	[TRAIN] Epoch=1/20, Step=20/704, loss=2.286431, acc1=0.156250, acc5=0.781250, lr=0.002500, time_each_step=0.08s, eta=0:17:49
2021-10-30 23:13:26 [INFO]	[TRAIN] Epoch=1/20, Step=30/704, loss=2.203204, acc1=0.218750, acc5=0.843750, lr=0.002500, time_each_step=0.07s, eta=0:15:34
2021-10-30 23:13:26 [INFO]	[TRAIN] Epoch=1/20, St

In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.cls.ShuffleNetV2_swish(num_classes=num_classes)
model.train(num_epochs=20,
            train_dataset=train_dataset,
            train_batch_size=32,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[10, 15, 17],
            save_interval_epochs=1,
            learning_rate=0.0025,
            early_stop=True,
            save_dir='output/shuffleNetV2_swish',
            use_vdl=True)

2021-10-31 00:46:19 [INFO]	Downloading ShuffleNetV2_x1_5_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ShuffleNetV2_x1_5_pretrained.pdparams


100%|██████████| 20810/20810 [00:00<00:00, 49662.43KB/s]


2021-10-31 00:46:19 [INFO]	Loading pretrained model from output/huffleNetV2_swish/pretrain/ShuffleNetV2_x1_5_pretrained.pdparams
2021-10-31 00:46:19 [WARNING]	[SKIP] Shape of pretrained params _fc.weight doesn't match.(Pretrained: [1024, 1000], Actual: [1024, 6])
2021-10-31 00:46:19 [WARNING]	[SKIP] Shape of pretrained params _fc.bias doesn't match.(Pretrained: [1000], Actual: [6])
2021-10-31 00:46:20 [INFO]	There are 280/282 variables loaded into ShuffleNetV2_x1_5.
2021-10-31 00:46:21 [INFO]	[TRAIN] Epoch=1/20, Step=10/704, loss=2.028213, acc1=0.218750, acc5=0.968750, lr=0.002500, time_each_step=0.08s, eta=0:19:20
2021-10-31 00:46:21 [INFO]	[TRAIN] Epoch=1/20, Step=20/704, loss=2.874357, acc1=0.093750, acc5=0.843750, lr=0.002500, time_each_step=0.05s, eta=0:10:52
2021-10-31 00:46:21 [INFO]	[TRAIN] Epoch=1/20, Step=30/704, loss=2.367817, acc1=0.250000, acc5=0.906250, lr=0.002500, time_each_step=0.05s, eta=0:10:48
2021-10-31 00:46:22 [INFO]	[TRAIN] Epoch=1/20, Step=40/704, loss=2.193158

# prune

In [6]:
#prune
model = pdx.load_model('output/MobileNetV3_large_ssld/best_model')

# Step 1/3: Analysis model of the sensitivity of each layer of the parameters in different cuts
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#analyze_sensitivity
model.analyze_sensitivity(
    dataset=eval_dataset, batch_size=32,save_dir='output/MobileNetV3_large_ssld/prune')


2021-10-31 01:44:40 [INFO]	Model[MobileNetV3_large_ssld] loaded.


2021-10-31 01:44:42,536-WARNING: Leaves ['t_344', 't_338', 't_337', 't_336', 't_331', 't_330', 't_317', 't_316', 't_312', 't_304', 't_290', 't_286', 't_285', 't_281', 't_280', 't_279', 't_266', 't_332', 't_260', 't_259', 't_255', 't_239', 't_235', 't_234', 't_291', 't_215', 't_210', 't_209', 't_208', 't_203', 't_202', 't_198', 't_197', 't_192', 't_261', 't_190', 't_184', 't_179', 't_174', 't_167', 't_162', 't_161', 't_156', 't_204', 't_155', 't_150', 't_149', 't_144', 't_143', 't_139', 't_138', 't_80', 't_113', 't_292', 't_137', 't_240', 't_151', 't_228', 't_68', 't_20', 't_160', 't_15', 't_318', 't_87', 't_241', 't_145', 't_347', 't_168', 't_191', 't_185', 't_26', 't_112', 't_118', 't_154', 't_233', 't_44', 't_82', 't_133', 't_86', 't_284', 't_254', 't_33', 't_39', 't_117', 't_166', 't_107', 't_186', 't_267', 't_214', 't_7', 't_230', 't_229', 't_67', 't_85', 't_311', 't_305', 't_81', 't_310', 't_196', 't_56', 't_253', 'fetch_0', 't_66', 't_180', 't_132', 't_1', 't_106', 't_2', 't_3', 

2021-10-31 01:44:45 [INFO]	Sensitivity analysis of model parameters starts...
2021-10-31 01:44:45 [INFO]	Start to evaluate(total_samples=5636, total_steps=177)...


2021-10-31 01:44:50,429-INFO: Pruning variable [conv1_weights] and its relatives ['conv1_weights', 'conv1_bn_scale', 'conv1_bn_offset', 'conv1_bn_mean', 'conv1_bn_variance', 'conv2_expand_weights', 't_22', 'conv2_linear_weights', 'conv2_linear_bn_scale', 'conv2_linear_bn_offset', 'conv2_linear_bn_mean', 'conv2_linear_bn_variance', 'conv3_expand_weights']


2021-10-31 01:44:50 [INFO]	Start to evaluate(total_samples=5636, total_steps=177)...


...

2021-10-31 02:06:50 [INFO]	Start to evaluate(total_samples=5636, total_steps=177)...


2021-10-31 02:06:55,250-INFO: pruned param: conv_last_weights; 0.8; loss=0.25313568115234375
2021-10-31 02:06:55,254-INFO: Save status into output/MobileNetV3_large_ssld/prune/model.sensi.data
2021-10-31 02:06:55,267-INFO: Pruning variable [conv_last_weights] and its relatives ['conv_last_weights', 'conv_last_bn_scale', 'conv_last_bn_offset', 'conv_last_bn_mean', 'conv_last_bn_variance', 'last_1x1_conv_weights']


2021-10-31 02:06:55 [INFO]	Start to evaluate(total_samples=5636, total_steps=177)...


2021-10-31 02:07:00,674-INFO: pruned param: conv_last_weights; 0.9; loss=0.4107753336429596
2021-10-31 02:07:00,677-INFO: Save status into output/MobileNetV3_large_ssld/prune/model.sensi.data


2021-10-31 02:07:00 [INFO]	Sensitivity analysis is complete. The result is saved at output/MobileNetV3_large_ssld/prune/model.sensi.data.


In [7]:
# Step 2/3: Tail the model according to the selected FLOPS reduction ratio
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#prune
model.prune(pruned_flops=.2)

2021-10-31 02:07:00 [INFO]	Pre-pruning FLOPs: 14288232.0. Pruning starts...


2021-10-31 02:07:00,827-INFO: Base FLOPs: 14631414.0
2021-10-31 02:07:00,829-INFO: Pruning variable [conv1_weights] and its relatives ['conv1_weights', 'conv1_bn_scale', 'conv1_bn_offset', 'conv1_bn_mean', 'conv1_bn_variance', 'conv2_expand_weights', 't_22', 'conv2_linear_weights', 'conv2_linear_bn_scale', 'conv2_linear_bn_offset', 'conv2_linear_bn_mean', 'conv2_linear_bn_variance', 'conv3_expand_weights']
2021-10-31 02:07:00,831-INFO: Pruning variable [conv2_expand_weights] and its relatives ['conv2_expand_weights', 'conv2_expand_bn_scale', 'conv2_expand_bn_offset', 'conv2_expand_bn_mean', 'conv2_expand_bn_variance', 'conv2_depthwise_weights', 'conv2_depthwise_bn_scale', 'conv2_depthwise_bn_offset', 'conv2_depthwise_bn_mean', 'conv2_depthwise_bn_variance', 'conv2_linear_weights']
2021-10-31 02:07:00,832-INFO: Pruning variable [conv3_expand_weights] and its relatives ['conv3_expand_weights', 'conv3_expand_bn_scale', 'conv3_expand_bn_offset', 'conv3_expand_bn_mean', 'conv3_expand_bn_var

2021-10-31 02:07:05 [INFO]	Pruning is complete. Post-pruning FLOPs: 11368734.0
2021-10-31 02:07:05 [WARNING]	Pruning the model may hurt its performance, retraining is highly recommended


In [8]:
# Step 3/3: Retrain the clipped model
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#train
# https://github.com/PaddlePaddle/PaddleX/tree/release/2.0.0/docs/parameters.md
model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    lr_decay_epochs=[4, 6, 8],
    learning_rate=0.025,
    pretrain_weights=None,
    save_dir='output/MobileNetV3_large_ssld/prune',
    use_vdl=True)

2021-10-31 02:07:06 [INFO]	[TRAIN] Epoch=1/10, Step=10/704, loss=1.486325, acc1=0.437500, acc5=1.000000, lr=0.025000, time_each_step=0.08s, eta=0:9:3
2021-10-31 02:07:06 [INFO]	[TRAIN] Epoch=1/10, Step=20/704, loss=1.356394, acc1=0.562500, acc5=0.875000, lr=0.025000, time_each_step=0.05s, eta=0:5:59
2021-10-31 02:07:07 [INFO]	[TRAIN] Epoch=1/10, Step=30/704, loss=1.394193, acc1=0.375000, acc5=1.000000, lr=0.025000, time_each_step=0.05s, eta=0:5:50
2021-10-31 02:07:07 [INFO]	[TRAIN] Epoch=1/10, Step=40/704, loss=1.663139, acc1=0.250000, acc5=0.968750, lr=0.025000, time_each_step=0.05s, eta=0:6:0
2021-10-31 02:07:08 [INFO]	[TRAIN] Epoch=1/10, Step=50/704, loss=1.606044, acc1=0.468750, acc5=0.906250, lr=0.025000, time_each_step=0.05s, eta=0:5:55
2021-10-31 02:07:08 [INFO]	[TRAIN] Epoch=1/10, Step=60/704, loss=1.466625, acc1=0.406250, acc5=0.968750, lr=0.025000, time_each_step=0.05s, eta=0:5:58
2021-10-31 02:07:09 [INFO]	[TRAIN] Epoch=1/10, Step=70/704, loss=1.716231, acc1=0.406250, acc5=0

In [5]:
#prune
model = pdx.load_model('output/shuffleNetV2_swish/best_model')

# Step 1/3: Analysis model of the sensitivity of each layer of the parameters in different cuts
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#analyze_sensitivity
model.analyze_sensitivity(
    dataset=eval_dataset, save_dir='output/shuffleNetV2_swish/prune')

# Step 2/3: Tail the model according to the selected FLOPS reduction ratio
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#prune
model.prune(pruned_flops=.2)

# Step 3/3: Retrain the clipped model
# API：https://github.com/PaddlePaddle/PaddleX/blob/release/2.0.0/docs/apis/models/classification.md#train
# https://github.com/PaddlePaddle/PaddleX/tree/release/2.0.0/docs/parameters.md
model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    lr_decay_epochs=[4, 6, 8],
    learning_rate=0.025,
    pretrain_weights=None,
    save_dir='output/shuffleNetV2_swish/prune',
    use_vdl=True)

2021-10-31 00:50:06 [INFO]	Model[ShuffleNetV2_swish] loaded.


2021-10-31 00:50:08,362-WARNING: Leaves ['t_458', 't_440', 't_439', 't_438', 't_433', 't_427', 't_421', 't_419', 't_413', 't_412', 't_408', 't_407', 't_403', 't_402', 't_401', 't_393', 't_388', 't_387', 't_376', 't_375', 't_369', 't_362', 't_361', 't_355', 't_349', 't_345', 't_344', 't_343', 't_340', 't_339', 't_336', 't_334', 't_327', 't_322', 't_321', 't_320', 't_423', 't_315', 't_351', 't_308', 't_300', 't_299', 't_296', 't_294', 't_290', 't_289', 't_284', 't_280', 't_275', 't_273', 't_270', 't_414', 't_269', 't_268', 't_256', 't_262', 't_397', 't_325', 't_249', 't_247', 't_244', 't_243', 't_242', 't_238', 't_237', 't_228', 't_222', 't_212', 't_211', 't_210', 't_206', 't_202', 't_197', 't_195', 't_192', 't_191', 't_186', 't_185', 't_184', 't_171', 't_170', 't_166', 't_165', 't_160', 't_263', 't_158', 't_258', 't_204', 't_236', 't_152', 't_150', 't_254', 't_145', 't_144', 't_143', 't_274', 't_140', 't_264', 't_139', 't_360', 't_138', 't_134', 't_133', 't_180', 't_132', 't_164', 't_12

2021-10-31 00:50:11 [INFO]	Sensitivity analysis of model parameters starts...
2021-10-31 00:50:11 [INFO]	Start to evaluate(total_samples=5636, total_steps=705)...


2021-10-31 00:50:28,796-INFO: Pruning variable [stage1_conv_weights] and its relatives ['stage1_conv_weights', 'stage1_conv_bn_scale', 'stage1_conv_bn_offset', 'stage1_conv_bn_mean', 'stage1_conv_bn_variance', 'stage_2_1_conv4_weights', 'stage_2_1_conv4_bn_scale', 'stage_2_1_conv4_bn_offset', 'stage_2_1_conv4_bn_mean', 'stage_2_1_conv4_bn_variance', 'stage_2_1_conv5_weights', 'stage_2_1_conv1_weights']
2021-10-31 00:50:28,801-INFO: change groups from 24 to 22 for stage_2_1_conv4_weights.


2021-10-31 00:50:28 [INFO]	Start to evaluate(total_samples=5636, total_steps=705)...


...

2021-10-31 01:37:32 [INFO]	Start to evaluate(total_samples=5636, total_steps=705)...


2021-10-31 01:37:48,692-INFO: pruned param: conv5_weights; 0.9; loss=0.2940647602081299
2021-10-31 01:37:48,695-INFO: Save status into output/shuffleNetV2_swish/prune/model.sensi.data


2021-10-31 01:37:48 [INFO]	Sensitivity analysis is complete. The result is saved at output/shuffleNetV2_swish/prune/model.sensi.data.
2021-10-31 01:37:48 [INFO]	Pre-pruning FLOPs: 16557712.0. Pruning starts...


2021-10-31 01:37:48,813-INFO: Base FLOPs: 16828768.0
2021-10-31 01:37:48,815-INFO: Pruning variable [stage1_conv_weights] and its relatives ['stage1_conv_weights', 'stage1_conv_bn_scale', 'stage1_conv_bn_offset', 'stage1_conv_bn_mean', 'stage1_conv_bn_variance', 'stage_2_1_conv4_weights', 'stage_2_1_conv4_bn_scale', 'stage_2_1_conv4_bn_offset', 'stage_2_1_conv4_bn_mean', 'stage_2_1_conv4_bn_variance', 'stage_2_1_conv5_weights', 'stage_2_1_conv1_weights']
2021-10-31 01:37:48,816-INFO: Pruning variable [stage_2_1_conv1_weights] and its relatives ['stage_2_1_conv1_weights', 'stage_2_1_conv1_bn_scale', 'stage_2_1_conv1_bn_offset', 'stage_2_1_conv1_bn_mean', 'stage_2_1_conv1_bn_variance', 'stage_2_1_conv2_weights', 'stage_2_1_conv2_bn_scale', 'stage_2_1_conv2_bn_offset', 'stage_2_1_conv2_bn_mean', 'stage_2_1_conv2_bn_variance', 'stage_2_1_conv3_weights']
2021-10-31 01:37:48,818-INFO: Pruning variable [stage_2_2_conv1_weights] and its relatives ['stage_2_2_conv1_weights', 'stage_2_2_conv1_bn

2021-10-31 01:37:52 [INFO]	Pruning is complete. Post-pruning FLOPs: 13219884.0
2021-10-31 01:37:52 [WARNING]	Pruning the model may hurt its performance, retraining is highly recommended
2021-10-31 01:37:53 [INFO]	[TRAIN] Epoch=1/10, Step=10/704, loss=1.435644, acc1=0.406250, acc5=1.000000, lr=0.025000, time_each_step=0.07s, eta=0:8:52
2021-10-31 01:37:53 [INFO]	[TRAIN] Epoch=1/10, Step=20/704, loss=2.771832, acc1=0.343750, acc5=0.968750, lr=0.025000, time_each_step=0.05s, eta=0:5:54
2021-10-31 01:37:54 [INFO]	[TRAIN] Epoch=1/10, Step=30/704, loss=2.510355, acc1=0.062500, acc5=0.875000, lr=0.025000, time_each_step=0.05s, eta=0:5:42
2021-10-31 01:37:54 [INFO]	[TRAIN] Epoch=1/10, Step=40/704, loss=2.907693, acc1=0.218750, acc5=0.968750, lr=0.025000, time_each_step=0.05s, eta=0:5:49
2021-10-31 01:37:55 [INFO]	[TRAIN] Epoch=1/10, Step=50/704, loss=2.472488, acc1=0.281250, acc5=0.875000, lr=0.025000, time_each_step=0.05s, eta=0:5:34
2021-10-31 01:37:55 [INFO]	[TRAIN] Epoch=1/10, Step=60/704,

In [9]:
model = pdx.load_model('output/MobileNetV3_large_ssld/best_model')

model.quant_aware_train(
    num_epochs=5,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    learning_rate=0.000025,
    save_dir='output/MobileNetV3_large_ssld/quant',
    use_vdl=True)

2021-10-31 02:14:12 [INFO]	Model[MobileNetV3_large_ssld] loaded.
2021-10-31 02:14:12 [INFO]	Preparing the model for quantization-aware training...
2021-10-31 02:14:13 [INFO]	Model is ready for quantization-aware training.
2021-10-31 02:14:14 [INFO]	[TRAIN] Epoch=1/5, Step=10/704, loss=0.640429, acc1=0.718750, acc5=1.000000, lr=0.000025, time_each_step=0.13s, eta=0:8:17
2021-10-31 02:14:15 [INFO]	[TRAIN] Epoch=1/5, Step=20/704, loss=0.857578, acc1=0.593750, acc5=1.000000, lr=0.000025, time_each_step=0.1s, eta=0:6:25
2021-10-31 02:14:16 [INFO]	[TRAIN] Epoch=1/5, Step=30/704, loss=1.051084, acc1=0.593750, acc5=1.000000, lr=0.000025, time_each_step=0.1s, eta=0:6:21
2021-10-31 02:14:17 [INFO]	[TRAIN] Epoch=1/5, Step=40/704, loss=0.817241, acc1=0.781250, acc5=1.000000, lr=0.000025, time_each_step=0.1s, eta=0:6:19
2021-10-31 02:14:18 [INFO]	[TRAIN] Epoch=1/5, Step=50/704, loss=0.690648, acc1=0.718750, acc5=1.000000, lr=0.000025, time_each_step=0.1s, eta=0:6:16
2021-10-31 02:14:19 [INFO]	[TRAI

# deploy

## according to the accuracy we only choose mobilenetv3

In [11]:
!paddlex --export_inference --model_dir=output/MobileNetV3_large_ssld/prune/best_model --save_dir=./inference_model_mobilenetv3

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."
[10-31 10:05:27 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --export_inference --model_dir=output/MobileNetV3_large_ssld/prune/best_model --save_dir=./inference_model_mobilenetv3
[10-31 10:05:27 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  

In [12]:
import paddlex as pdx
predictor = pdx.deploy.Predictor('inference_model_mobilenetv3/inference_model')
result = predictor.predict(img_file='work/angry/im0.png',
                           warmup_iters=100,
                           repeats=100)

2021-10-31 10:08:57 [INFO]	Model[MobileNetV3_large_ssld] loaded.
------------------ Inference Time Info ----------------------
total_time(ms): 818.0, img_num: 0, batch_size: 0.0
average latency time(ms): 8.18, QPS: 122.249389
preprocess_time_per_im(ms): 0.40, inference_time_per_batch(ms): 7.70, postprocess_time_per_im(ms): 0.10


In [16]:
import paddlex as pdx
test_jpg = 'work/angry/im11.png'
predictor = pdx.deploy.Predictor('inference_model_mobilenetv3/inference_model')
result = predictor.predict(test_jpg)
print("Predict Result: ", result)

2021-10-31 10:10:49 [INFO]	Model[MobileNetV3_large_ssld] loaded.
Predict Result:  [{'category_id': 0, 'category': 'angry', 'score': 0.6732684}]
